In [2]:
import pandas as pd

from sklearn.neighbors import NearestNeighbors
import numpy as np
import implicit
from scipy.sparse import csr_matrix, save_npz
from tqdm import trange
import nmslib

In [3]:
%%time
data = pd.read_csv("data/24tv_channel_2024_11_11.csv")
data.shape


FileNotFoundError: [Errno 2] No such file or directory: 'data/24tv_channel_2024_11_11.csv'

In [4]:
data.profile.nunique()

362230

In [5]:
data.user.nunique()

374502

In [6]:
data.is_guest.value_counts()

is_guest
0                  8079224
1                  2325663
0                    24802
1                     7965
Nullable(UInt8)          1
Name: count, dtype: int64

In [18]:
data.columns

Index(['id', 'uuid', 'external_stat_id', 'user', 'profile', 'device',
       'device_type', 'channel', 'schedule', 'history', 'timestamp',
       'duration', 'streamer', 'stream_type', 'client_ip', 'provider',
       'is_guest', 'user_state', 'application_type', 'is_live', 'started_at',
       'ended_at', 'preview', 'session_id', 'access_token', 'app_version',
       'cause', 'content', 'top_parent'],
      dtype='object')

In [9]:
list(data.duration).count('0')


19685

In [20]:
data.duration.head(50)

0     UInt32
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19       217
20         0
21       117
22         0
23         0
24        21
25         0
26       113
27         0
28         0
29         0
30       706
31         0
32         0
33         0
34        30
35         0
36        37
37         0
38     86597
39         0
40         0
41         0
42        92
43         0
44        14
45         0
46         0
47        37
48         0
49       191
Name: duration, dtype: object

In [23]:
data.iloc[39, :]

id                                       1057300220916611289
uuid                    223f70ed-783a-45d1-9ddc-151776eefaf5
external_stat_id                                          \N
user                                                10748167
profile                 690fb585-2144-4553-a579-fbdd2eb59f70
device                  f30bfce8-01f4-40e1-91c2-1e97513c1800
device_type                                              stb
channel                                                 3946
schedule                                    0039461731345900
history                                                   \N
timestamp                                                  0
duration                                                   0
streamer                                                  36
stream_type                                           config
client_ip                                    212.113.114.113
provider                                                   2
is_guest                

In [3]:
from datetime import datetime

def time_difference(datetime_str1, datetime_str2):
    # Преобразуем строки в объекты datetime
    dt1 = datetime.strptime(datetime_str1, '%Y-%m-%d %H:%M:%S')
    dt2 = datetime.strptime(datetime_str2, '%Y-%m-%d %H:%M:%S')
    
    # Вычисляем разницу во времени
    difference = abs((dt2 - dt1).total_seconds())
    
    return difference

# Пример использования
datetime1 = '2024-11-11 19:33:51'
datetime2 = '2024-11-11 20:00:00'
print(time_difference(datetime1, datetime2))  # Вывод: 1569.0


1569.0


In [4]:
dats = data[ data['started_at'] != data['ended_at']]

In [12]:
dats.profile.nunique()

232555

In [7]:
dats.head()

,id,uuid,external_stat_id,user,profile,device,device_type,channel,schedule,history,...,is_live,started_at,ended_at,preview,session_id,access_token,app_version,cause,content,top_parent
0,UInt64,Nullable(UUID),Nullable(UUID),UInt64,Nullable(FixedString(36)),Nullable(FixedString(36)),Nullable(String),UInt32,Nullable(FixedString(32)),Nullable(UInt64),...,Nullable(UInt8),DateTime,Nullable(DateTime),Nullable(UInt8),Nullable(UUID),Nullable(String),Nullable(UInt32),Nullable(String),Nullable(UInt64),Nullable(UInt64)
5,1057300220071140962,fb38bb95-d0b5-4e75-b516-12550b8927ee,\N,18159553,9ec826f2-6f15-4b3c-8bce-b9475370ff3f,a72e7af9-4ba8-402e-9958-45bc4c4506d0,stb,3572,0035721731348000,1057300601860866919,...,1,2024-11-11 21:00:29,2024-11-11 21:00:43,0,fb38bb95-d0b5-4e75-b516-12550b8927ee,04752dd5cf844177deb6b7522d5c4740ea0235d5,3626,switch,1056179973006797522,546310946419639028
19,1057300220486501819,cf42f37d-a9b1-4b04-88fd-1f6dee6d0e6e,\N,12588293,20bfa900-b2d9-4d5a-ba19-004c8a75b750,d08a306c-fc5e-48d4-beb0-7423bc35cbfd,stb,228,0002281731348000,1057300457480348641,...,1,2024-11-11 21:02:55,2024-11-11 21:08:23,0,cf42f37d-a9b1-4b04-88fd-1f6dee6d0e6e,5f4e3c200ffdcfada3caa46a252abbe35d9c1511,3626,switch,764028576874067336,1013522749525350272
21,1057300220516557250,814b4686-2106-4a65-a6ec-86c29e2315e5,\N,16969810,494fef0a-b759-422b-b0be-58bfb4acd7e4,51af3570-62dc-46a1-b620-c7b8e2c2fc60,phone,49,0000491731346800,1057300918383682787,...,1,2024-11-11 20:58:56,2024-11-11 21:03:05,0,814b4686-2106-4a65-a6ec-86c29e2315e5,1a7c7f87adfd3677e04e961a0703f58644b308f3,\N,\N,546313275189494771,\N
24,1057300220532719674,1b586896-4a25-42b4-8712-543bd2d8bbf6,\N,13476238,54a2be20-b55a-44c4-b748-05be736593a1,aefba572-5e1e-4cd9-b182-c4be315c7f9d,tv,149,0001491731687600,1057299500847387475,...,0,2024-11-11 20:58:24,2024-11-11 20:58:56,0,964c99c2-2b5d-4baf-9633-3eae47b0cdd4,90eefa8af65d9182c0f42c6579bd4e86c27106b9,324514,switch,791219009014729866,\N


In [5]:
def time_difference_row(row):
    return time_difference(row['ended_at'], row['started_at'])

In [7]:
dats = dats.drop(index = 0)

In [29]:
data  = data.iloc[:1000000, :]

In [31]:
data.shape

(1000000, 29)

In [8]:
dats['time_diff'] = dats.apply(time_difference_row, axis=1)


In [10]:
dats.shape

(5010311, 30)

In [28]:
a.count(0)

7221

In [75]:
data.video

0         UInt32
1              0
2              0
3              0
4              0
          ...   
9996           0
9997           0
9998     1372271
9999           0
10000          0
Name: video, Length: 10001, dtype: object

In [59]:
data.columns

Index(['id', 'uuid', 'external_stat_id', 'user', 'profile', 'device',
       'device_type', 'channel', 'schedule', 'history', 'timestamp',
       'duration', 'streamer', 'stream_type', 'client_ip', 'provider',
       'is_guest', 'user_state', 'application_type', 'is_live', 'started_at',
       'ended_at', 'preview', 'session_id', 'access_token', 'app_version',
       'cause', 'content', 'top_parent'],
      dtype='object')

In [4]:
da = pd.read_csv("data/history.csv", sep='|')
da


,profile,content_id,top_parent_id,seconds,viewed_at,duration,video_id,episode_id,program_id,object_id,content_type_id,percent_view,expired_at,user
0,724d5ce2-0599-46e3-b1ad-0cd41e0cfd99,689642421974512523,546316123402928564,1614,2024-08-26 12:22:52,NaN,0,0,0,2.171720e+12,20,0,NaN,NaN
1,46095f79-20cd-4e61-86df-18efaf70badf,746615188702666611,654994236770384070,1235,2024-07-07 12:50:35,NaN,0,0,0,3.769172e+13,20,0,NaN,NaN
2,c1cce0b3-e344-4980-bc4b-82960d11f05e,546327070486037003,0,590,2024-08-31 10:09:50,NaN,0,0,0,7.172025e+10,20,0,NaN,NaN
3,f85d3865-2999-4b82-86f9-0e52ad1d7ea9,546311973344641608,562690200531108268,1952,2024-07-08 16:25:02,NaN,0,0,0,1.011517e+14,20,0,NaN,NaN
4,617ac18b-ddcb-4076-9bc5-5da844472962,558372338278596996,726674929764327360,6460,2024-07-10 23:17:37,NaN,0,0,0,1.011517e+14,20,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,02a3dd82-5d84-4692-aee5-c0b0a1656668,1009112971570859916,546325163948376325,3590,2024-07-13 11:03:38,NaN,0,0,0,8.172025e+10,20,0,NaN,NaN
1996,8f1f6c9d-e6fc-438d-aa5e-eb86aded63a5,562690202083001302,562690200531108268,3096,2024-07-08 03:14:46,NaN,0,0,0,1.011517e+14,20,0,NaN,NaN
1997,07b6eef7-eb95-48a3-b2ff-e195e9700723,581457819828486821,575374350320927100,3495,2024-07-15 17:02:41,NaN,0,0,0,2.811720e+12,20,0,NaN,NaN
1998,4d1c2466-b2d5-4e67-b002-9570337ce3a9,560146620746826604,546310935006937179,3571,2024-07-10 17:13:46,NaN,0,0,0,3.579172e+13,20,0,NaN,NaN


In [35]:
da.percent_view.unique()

array([0], dtype=int64)

In [45]:
data.shape

(473434, 30)

In [44]:
data = data.drop_duplicates()

In [39]:
data = data[data['time_diff'] !=0.0]


In [52]:
data.time_diff.shape

(473434,)

In [187]:
dats.shape

(5010311, 30)

In [9]:
row_indices = pd.factorize(dats['profile'])[0]
col_indices = pd.factorize(dats['content'])[0]

# Создаем CSR-матрицу
csr_result = csr_matrix((dats['time_diff'], (row_indices, col_indices)), 
                         shape=(len(dats['profile'].unique()), len(dats['content'].unique())))

In [191]:
dic = dats.iloc[row_indices, :]['profile']

In [196]:
dic.iloc[1486]

'138a3a5e-ab48-489c-94e5-90f164043224'

5         9ec826f2-6f15-4b3c-8bce-b9475370ff3f
19        20bfa900-b2d9-4d5a-ba19-004c8a75b750
21        494fef0a-b759-422b-b0be-58bfb4acd7e4
24        54a2be20-b55a-44c4-b748-05be736593a1
26        aabcbced-1e77-4372-86ce-e14841bba37a
                          ...                 
25785     cff9f87c-64d4-4105-a1e4-279df2e1fa3d
65772     0e7d2889-492a-4eeb-b844-738c04f100b0
85528     3f670131-9806-40cc-8f74-2f6777beafec
85280     3b705524-82a6-4af6-b5e5-2f648b019120
122723    afcaa5fa-8902-41be-a56b-68249a20d1b1
Name: profile, Length: 5010311, dtype: object

In [17]:
csr_result

<232554x31916 sparse matrix of type '<class 'numpy.float64'>'
	with 2513965 stored elements in Compressed Sparse Row format>

In [48]:
data.set_index(['profile', 'content', 'time_diff'], append=True)


,,,,id,uuid,external_stat_id,user,device,device_type,channel,schedule,history,timestamp,...,application_type,is_live,started_at,ended_at,preview,session_id,access_token,app_version,cause,top_parent
,profile,content,time_diff,,,,,,,,,,,,,,,,,,,,,
5,9ec826f2-6f15-4b3c-8bce-b9475370ff3f,1056179973006797522,14.0,1057300220071140962,fb38bb95-d0b5-4e75-b516-12550b8927ee,\N,18159553,a72e7af9-4ba8-402e-9958-45bc4c4506d0,stb,3572,0035721731348000,1057300601860866919,0,...,android-tv,1,2024-11-11 21:00:29,2024-11-11 21:00:43,0,fb38bb95-d0b5-4e75-b516-12550b8927ee,04752dd5cf844177deb6b7522d5c4740ea0235d5,3626,switch,546310946419639028
19,20bfa900-b2d9-4d5a-ba19-004c8a75b750,764028576874067336,328.0,1057300220486501819,cf42f37d-a9b1-4b04-88fd-1f6dee6d0e6e,\N,12588293,d08a306c-fc5e-48d4-beb0-7423bc35cbfd,stb,228,0002281731348000,1057300457480348641,0,...,android-tv,1,2024-11-11 21:02:55,2024-11-11 21:08:23,0,cf42f37d-a9b1-4b04-88fd-1f6dee6d0e6e,5f4e3c200ffdcfada3caa46a252abbe35d9c1511,3626,switch,1013522749525350272
21,494fef0a-b759-422b-b0be-58bfb4acd7e4,546313275189494771,249.0,1057300220516557250,814b4686-2106-4a65-a6ec-86c29e2315e5,\N,16969810,51af3570-62dc-46a1-b620-c7b8e2c2fc60,phone,49,0000491731346800,1057300918383682787,0,...,android-mobile,1,2024-11-11 20:58:56,2024-11-11 21:03:05,0,814b4686-2106-4a65-a6ec-86c29e2315e5,1a7c7f87adfd3677e04e961a0703f58644b308f3,\N,\N,\N
24,54a2be20-b55a-44c4-b748-05be736593a1,791219009014729866,32.0,1057300220532719674,1b586896-4a25-42b4-8712-543bd2d8bbf6,\N,13476238,aefba572-5e1e-4cd9-b182-c4be315c7f9d,tv,149,0001491731687600,1057299500847387475,1731687600,...,webkit,0,2024-11-11 20:58:24,2024-11-11 20:58:56,0,964c99c2-2b5d-4baf-9633-3eae47b0cdd4,90eefa8af65d9182c0f42c6579bd4e86c27106b9,324514,switch,\N
26,aabcbced-1e77-4372-86ce-e14841bba37a,1056189497323061859,153.0,1057300220587773853,efb8d28f-0e13-4233-b19f-da9b9d3e69d0,\N,11952324,f23c963f-1538-4c36-aab0-02ee79275ff3,stb,4833,0048331731346800,1057300570463967671,0,...,android-tv,1,2024-11-11 20:58:56,2024-11-11 21:01:29,0,efb8d28f-0e13-4233-b19f-da9b9d3e69d0,86aa194e4a653dde912310fb5e2044c8f6543591,3626,\N,546309885382361316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,dd8b06bf-230e-421f-9a84-d7dcf77a0d88,1056795893175426952,210.0,1057022950765559371,f033cfd0-e91e-4428-9f45-c99186483d68,\N,14245328,cf15a180-ccb7-4145-99ca-c62262d9f328,stb,163,0001631731288900,1057023125260951213,0,...,android-tv,1,2024-11-11 04:35:11,2024-11-11 04:38:41,0,f033cfd0-e91e-4428-9f45-c99186483d68,f6dabe11a63c4ba221669b91cbde9fcd20d05e4c,3619,switch,\N
999996,5fd84fe1-432e-4650-8004-005234f2f84b,\N,31.0,1057022950797333068,ba6aa69a-c882-407a-914c-33500a7f821e,\N,216316,7f148fcb-213a-4912-8624-87372a134dc3,tv,115,\N,\N,1731897600,...,webkit,0,2024-11-11 02:10:12,2024-11-11 02:10:43,0,f9f5aeb8-c0a0-4a65-8422-472a13b33962,abddd28e0d864164f3a94ecafc82b5c16d1be0f9,324514,null_schedule,546332049783390964
999998,53a5c747-7d84-4fea-a56d-e171a6bb11e1,\N,30.0,1057022950994326653,b3ad21ac-98f6-41d7-832a-e594de0e90ee,\N,873353,d5f83ff8-e0e7-47a3-b841-c6000b4109c1,stb,116,\N,\N,1731890400,...,android-tv,0,2024-11-11 02:10:13,2024-11-11 02:10:43,0,82ae744e-de65-4ced-95f3-4896b36fd7ab,c848ca4252c690e7e001d6169ed7db856fb631f9,3619,null_schedule,\N


In [55]:
net = net.T



In [43]:
2000/(802*1555)

0.0016037077723697188

In [53]:
data_joined = da.merge(data, on = 'profile')

In [52]:
data.dtypes


id                  object
uuid                object
external_stat_id    object
user                object
profile             object
device              object
device_type         object
channel             object
schedule            object
history             object
timestamp           object
duration            object
streamer            object
stream_type         object
client_ip           object
provider            object
is_guest            object
user_state          object
application_type    object
is_live             object
started_at          object
ended_at            object
preview             object
session_id          object
access_token        object
app_version         object
cause               object
content             object
top_parent          object
dtype: object

In [55]:
data_joined

,profile,content_id,top_parent_id,seconds,viewed_at,duration_x,video_id,episode_id,program_id,object_id,...,is_live,started_at,ended_at,preview,session_id,access_token,app_version,cause,content,top_parent
0,7703a0ad-9833-4b24-ac34-fd526793ddd8,558370347452203662,546316688543449479,3634,2024-07-06 10:34:10,NaN,0,0,0,2.821720e+12,...,1,2024-11-11 19:34:36,2024-11-11 19:34:36,0,53a19e49-9d1c-4744-8d50-85b59455cf57,783e5f2c28df525840102f58153466e9d2d89707,3626,\N,572469207506092136,546320148504511064
1,0f1ee1f8-ab2b-43bf-9edc-e3dcc233d1ff,547488304073801949,547488284595454673,269,2024-07-06 10:34:30,NaN,0,0,0,2.821720e+12,...,1,2024-11-11 19:34:41,2024-11-11 19:34:41,0,ffa91586-0653-4e34-9962-3c8fed4bd44e,01a683c8d55e674c90f28c03185b6f48d42ecd81,3545,\N,547488060258910692,546316688543449479
2,2eee53a4-8efc-4b6d-a34c-ea2777957f15,1006635702612305797,986730752877548594,2069,2024-07-06 10:34:30,NaN,0,0,0,8.172025e+10,...,0,2024-11-11 18:52:32,2024-11-11 19:10:31,0,7269f1c9-faf9-45db-a398-391d7439738a,e1e86213599ab8ed25389e53f60962013cd1276d,3619,\N,1057122840058095053,546311727910748959
3,b685bba3-2eda-4982-95c5-c181600f227c,546330729785590263,546314524995617441,523,2024-07-06 10:42:55,NaN,0,0,0,2.181720e+12,...,0,2024-11-11 19:34:04,2024-11-11 19:34:04,0,95502cbd-c040-4cfc-9b8b-7ff2d5f046ad,5b893bd20ed964171260a4063f0483c3b1b4de80,3619,\N,546309805774472070,\N
4,a65dd797-ef21-4b24-87c5-3f68b8d28536,546310393270633467,546310015753913209,9453,2024-07-06 10:34:35,NaN,0,0,0,6.172047e+10,...,0,2024-11-11 19:33:35,2024-11-11 19:34:03,0,ca34ff4e-41eb-455d-9ac0-08c2b4aae2bd,cf1071fb71af7827a9fff793300da5e434d0a900,3626,null_schedule,\N,1053706819639757286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,e250787e-e848-4bb2-ac14-afb00d0fec6b,818785867532946269,756600418471587689,454,2024-07-06 10:43:04,NaN,0,0,0,8.000000e+00,...,1,2024-11-11 19:34:32,2024-11-11 20:14:25,0,a61c8b4b-55ac-4a3f-a30c-6fa59b5bde14,c72aa75fd89863c2b17ab0a80d6baeb945045c8d,3619,\N,1057122865634179579,546319679417746392
77,82d87d0a-b38b-444f-9b7c-4154691b7783,550023913178923841,575374350320927100,8016,2024-07-06 10:41:13,NaN,0,0,0,2.811721e+12,...,0,2024-11-11 19:27:23,2024-11-11 19:34:01,0,fe4f036b-481d-4074-95b9-52ad200c6f16,626be56c8cc08d22aadc25d01e3c88183cfe0b43,300278,\N,600951211545330430,600951193119752387
78,591fd805-3eaa-44e4-8d05-72d94adb3f63,771387794690015717,568574330028426195,7447,2024-07-14 00:40:28,NaN,0,0,0,2.981720e+12,...,0,2024-11-11 18:52:35,2024-11-11 18:52:54,0,9698514c-37c0-4a42-942d-c111c6792562,c1a3da3029e23bb604ee169e8b58d8c46b5674d0,\N,\N,608526204835725607,\N
79,4008b270-4bcc-48b2-bb76-d736ad2ec547,845894030071435424,546310935006937179,9658,2024-07-06 10:36:21,NaN,0,0,0,6.172051e+10,...,0,2024-11-11 18:52:34,2024-11-11 18:53:44,0,d37d50d7-c0a2-43e3-aa8d-1a7d68f0576a,986e3d5c849d300be658dfb5545cc3ffbad0af85,324514,\N,927642662724954291,925165178208766737


In [25]:
dats.time_diff

1           0.0
2           0.0
3           0.0
4           0.0
6           0.0
           ... 
10437640    0.0
10437641    0.0
10437642    0.0
10437643    0.0
10437651    0.0
Name: time_diff, Length: 5427343, dtype: float64

In [56]:
net = net.fillna(0)

In [65]:
net = net.astype('int8')


In [67]:
net.memory_usage().sum()

2109375102

In [72]:
net = csr_matrix(net)


In [10]:
def remove_sparse_rows(csr_mat, min_nonzero):
    # Находим индексы строк, которые нужно сохранить
    row_mask = csr_mat.getnnz(axis=1) >= min_nonzero
    # Создаем новую CSR-матрицу с оставшимися строками
    return csr_mat[row_mask]

# Удаляем строки с менее чем 10 ненулевыми значениями
filtered_nett = remove_sparse_rows(csr_result, 10)

In [19]:
filtered_net.shape

(76446, 31916)

In [11]:
filtered_net = filtered_nett.tolil()

In [12]:
filtered_net

<76446x31916 sparse matrix of type '<class 'numpy.float64'>'
	with 1920530 stored elements in List of Lists format>

In [ ]:
index = nmslib.init(method='hnsw', space='l2')
index.addDataPointBatch(filtered_nett.toarray())
index.createIndex({'post': 2}, print_progress=True)

In [14]:
import matplotlib.pyplot as plt

In [13]:
np.median(csr_result.getnnz(axis=1))

6.0

In [15]:
distances, indices = nbrs.kneighbors(net)


In [19]:
indices

array([[   0, 1390,  280,   11, 1332],
       [   1,  796,  141,  777,    5],
       [   2, 1270,  318,  338,  623],
       ...,
       [1552, 1270,  338,  318,  623],
       [1553, 1282, 1270,  318,  338],
       [1554, 1254,  358, 1208,  441]], dtype=int64)

In [183]:
def predict(i, n_neighb):
    d, ind = nbrs.kneighbors(filtered_net[i], n_neighbors=n_neighb+1)
    print(filtered_net[i])
    print('='*30)
    for i in ind[0][1:]:
        print(i)
        #print(np.nonzero(filtered_net[i])[1])

In [184]:
predict(1488, 5)

  (0, 70)	1190.0
  (0, 84)	336.0
  (0, 89)	568.0
  (0, 96)	69.0
  (0, 179)	3785.0
  (0, 183)	836.0
  (0, 195)	1902.0
  (0, 202)	695.0
  (0, 272)	3267.0
  (0, 318)	31.0
  (0, 375)	3445.0
  (0, 397)	1201.0
  (0, 460)	1612.0
  (0, 474)	123.0
  (0, 496)	24.0
  (0, 501)	9231.0
  (0, 583)	434.0
  (0, 660)	194.0
  (0, 681)	1860.0
  (0, 1506)	513.0
  (0, 1891)	179.0
  (0, 2124)	526.0
  (0, 2158)	248.0
  (0, 2650)	372.0
  (0, 3344)	124.0
  (0, 4053)	2812.0
  (0, 6105)	117.0
  (0, 7830)	527.0
  (0, 7875)	4030.0
  (0, 7897)	1178.0
  (0, 9541)	539.0
  (0, 9830)	278.0
  (0, 11128)	117.0
  (0, 26860)	92.0
70439
67347
59247
37717
63649


In [24]:
net = filtered_net.copy()

In [25]:
net

<76446x31916 sparse matrix of type '<class 'numpy.float64'>'
	with 1920530 stored elements in List of Lists format>

In [26]:
def predict_cnt(i, n_neighb, net):
    d, ind = nbrs.kneighbors(net[i], n_neighbors=n_neighb+1)
    a = net[ind[0][0]]
    for ii in ind[0][1:]:
        a+=net[ii]
        #print('-'*30)
    for j in range(a.shape[1]):
        if a[0, j] != 0:  # Добавляем только ненулевые значения
            net[i, j] = a[0, j]
    


In [27]:
%%time
for i in trange(100):
    predict_cnt(i, 5, net)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.68it/s]

CPU times: total: 17.6 s
Wall time: 17.6 s


In [171]:
filtered_net.shape

(76446, 31916)

In [174]:
save_npz("csr_user_item.npz", filtered_net)

In [ ]:
def predict_cnt(i, n_neighb):
    d, ind = nbrs.kneighbors(filtered_net[i], n_neighbors=n_neighb+1)
    print(filtered_net[ind[0][0]].nnz)
    print('='*30)
    a = filtered_net[ind[0][1]]
    for i in ind[0][2:]:
        a+=filtered_net[i]
        #print('-'*30)
    print(a.nnz)

In [84]:
print(net[5,:])

  (0, 12831)	-119


In [35]:
import numpy as np
import implicit

# Пример матрицы взаимодействий
R = np.array(filtered_net)

# Преобразуем в формат, подходящий для библиотеки
R = R.astype(np.float32)
model = implicit.als.AlternatingLeastSquares(factors=5, regularization=0.1, iterations=10)

# Обучаем модель
model.fit(R)

# Предсказание
user_id = 0
recommended = model.recommend(user_id, R[user_id])
print(recommended)


ValueError: setting an array element with a sequence.

In [83]:
%%cmd
pip install implicit

Microsoft Windows [Version 10.0.19045.5131]
(c) ��௮��� �������� (Microsoft Corporation). �� �ࠢ� ���饭�.

C:\jupyter\proj\RecSys-24TV>pip install implicit
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 750.8/750.8 kB 5.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



C:\jupyter\proj\RecSys-24TV>

In [48]:
%%time


a=0
for i in range(10):
    R = filtered_net

    # Создаем и обучаем модель
    model = implicit.als.AlternatingLeastSquares(factors=10, regularization=0.1, iterations=10)
    model.fit(R)

    
    # Рекомендуем элементы для пользователя
    user_id = 59247
    recommended, _ = model.recommend(user_id, R[user_id])
    a += (70439 in recommended or 67347 in recommended or 59247 in recommended)
    print(recommended)

  0%|          | 0/10 [00:00<?, ?it/s]

[  51  318  183    8 1028 1432  246 4063  234  147]


  0%|          | 0/10 [00:00<?, ?it/s]

[  51  318    8  183   10 1432 4063 1028  147  202]


  0%|          | 0/10 [00:00<?, ?it/s]

[  10   51  318  183 1028    8 1432 4063  246  147]


  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [46]:
a

0

In [ ]:
[ 208 1247 1083 1889  736 1910 9240 4073 2887 4056]
